# Taipei food and beverage industry insight
---
Market Sizing, Competition, Barrier of Entry, and Recommendation
<br>
<br>
11 Oct 2020
<br>
<br>
<br>

<div class="subtitle">
        *all of the chart are interactive, hover your mouse and you can see data detail<br>
        *details about the number are in <a href="https://github.com/curiouslaw/taipei_food_industry_insight/tree/master/analysis">analysis folder</a><br>
        *details about the code (web scarping, data pipeline, etc) are in <a href="https://github.com/curiouslaw/taipei_food_industry_insight/tree/master/">github page</a>
</div>


In [ ]:
import os
import json
import warnings
from functools import partial
from typing import Union
from io import StringIO
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets.embed import embed_minimal_html, dependency_state
from shapely.geometry import Point, MultiPoint
from IPython.display import display, Markdown, HTML

from shared_lib import data_analysis
from data_analysis.lib.plotly_helper import (add_chart_title,
                                             add_chart_annotation,
                                             add_chart_bottom_annotation,
                                             set_layout_size)

BASE_DIR = os.path.dirname(os.path.abspath('__file__'))

plotly_default_config_chart = dict(
    responsive=False,
    displayModeBar=True,
    displaylogo=False,
    modeBarButtonsToRemove=['zoomIn2d', 'zoomOut2d', 'zoom2d', 'select2d', 
                            'lasso2d', 'toggleSpikelines', 'hoverClosestGl2d', 'toImage'],
)

plotly_default_config_geo = dict(
    responsive=False,
    displayModeBar=True,
    displaylogo=False,
    modeBarButtonsToRemove=['select2d', 'lasso2d', 'toImage'])

plotly_default_layout_size = dict(
    width=800,
    height=450,
)

class Agenda:
    def __init__(self, agenda_list: list):
        self.agenda_list = agenda_list
        self.higlight_wrap = ('<span style="font-size:200%; color:black">'
                              '<a href="#/{}" style="text-decoration:inherit;color:inherit">', '</a></span>')
        self.non_higlight_wrap = ('<span style="font-size:100%; color:grey">'
                                  '<a href="#/{}" style="text-decoration:inherit;color:inherit">', '</a></span>')
        
    def display(self) -> None:
        display(Markdown('\n\n'.join(self.agenda_list)))
        
    def highlight(self, index: int) -> None:
        h_wrap = self.higlight_wrap
        nh_wrap = self.non_higlight_wrap
        markdown_list = []
        
        for i, x in enumerate(self.agenda_list):
            if i == index:
                markdown_list.append(h_wrap[0].format(x[1]) + x[0] + h_wrap[1])
            else:
                markdown_list.append(nh_wrap[0].format(x[1]) + x[0] + nh_wrap[1])
        
        display(Markdown('\n\n'.join(markdown_list)))

current_agenda = Agenda([('Know more before starting a business',1),
                         ('The opportunity is huge on around Mid area',5),
                         ('Less competition in Mid, Mid-North, and Mid-East',9),
                         ('Good ROI on Mid-North and Mid-East',13),
                         ('Recommendation',16)])

In [ ]:
%%HTML
<style>

    div.output{
        flex-direction:row;
        align-items:center;
        justify-content:center;
    }
    
    div.output_area{
        flex:1;
        align-self: flex-start;
        align-items:center;
        justify-content:center;
    }
    
    div.output_subarea{
        padding:0;
        max-width:100%;
    }

    div.plotly-graph-div,
    div.js-plotly-plot {
        margin: 0 auto;
    }
    
    div.subtitle {
        font-size:70%;
        font-weight:normal;
        color:grey;
        margin-top: 10px;
    }

    p {
      color:grey
    }

    span.h_text {
     font-size:200%; 
     color:black;
    }
    
    hr {
        border: 1px solid darkgrey;
    }
</style>

In [ ]:
# load plotly javascript into browser
fig = go.Figure()
fig.update_layout(dict(yaxis={'visible':False},xaxis={'visible':False}))
set_layout_size(fig,10,10)
fig.show(config=dict(displayModeBar=False))

In [ ]:
# setup path and load data
data_mart_dirpath = os.path.join(BASE_DIR, 'data', 'aggregated-data_mart')
data_mart_table_filename_list = [x for x in os.listdir(data_mart_dirpath) if x.endswith('.csv')]
data_mart_filepath_list = [os.path.join(data_mart_dirpath, x) for x in data_mart_table_filename_list]

taipei_geojson_filename_list = [x for x in os.listdir(data_mart_dirpath) if x.endswith('.geojson')]
assert len(taipei_geojson_filename_list) == 2

taipei_geojson_village_filename = [x for x in taipei_geojson_filename_list if 'village' in x][0]
taipei_geojson_village_filepath = os.path.join(data_mart_dirpath, taipei_geojson_village_filename)

taipei_geojson_township_filename = [x for x in taipei_geojson_filename_list if 'township' in x][0]
taipei_geojson_township_filepath = os.path.join(data_mart_dirpath, taipei_geojson_township_filename)

key_index = 'village_code'

for index, path in enumerate(data_mart_filepath_list):
    if index==0:
        taipei_data_df = pd.read_csv(path).set_index(key_index, drop=False)
        taipei_data_df.index = taipei_data_df.index.astype(str)
    else:
        _new_df = pd.read_csv(path).set_index('village_code')
        _new_df.index = _new_df.index.astype(str)
        
        same_columns = [x for x in _new_df if x in taipei_data_df.columns]
        _new_df = _new_df.drop(same_columns, axis=1)
        
        taipei_data_df = pd.merge(taipei_data_df, _new_df,
                                  how='left', left_index=True, right_index=True)

taipei_data_gdf = gpd.read_file(taipei_geojson_village_filepath).set_index('village_code')
taipei_data_gdf.index = taipei_data_gdf.index.astype(str)

taipei_data_df = pd.merge(taipei_data_df, taipei_data_gdf,
                          how='left', left_index=True, right_index=True)

taipei_data_gdf = gpd.GeoDataFrame(taipei_data_df)

taipei_data_geojson = json.loads(taipei_data_gdf.geometry.to_json())

taipei_township_gdf = gpd.read_file(taipei_geojson_township_filepath)
taipei_township_geojson = json.loads(taipei_township_gdf.geometry.to_json())

In [ ]:
# set default charting
add_chart_title = partial(add_chart_title, title_margin=False)

def generate_custom_colour(color_list: list) -> list:
    custom_colour_list = color_list
    colour_range = list(zip(np.linspace(0,1,len(custom_colour_list)), custom_colour_list))
    colour_range.insert(0,(-0.01,'grey'))

    return colour_range


def default_label_format(label: str) -> str:
    new_label = label.replace('_',' ')
    new_label_list = new_label.split(' ')

    processed_label_list = []
    for w in new_label_list:
        if all([c.isupper() for c in w]):
            processed_label_list.append(w)
        else:
            processed_label_list.append(w.title())

    processed_label = (' ').join(processed_label_list)

    return processed_label


def default_choropleth_mapbox_fig(geo_data_frame: gpd.GeoDataFrame,
    scale_column_name: str, scale_range: tuple = {}, color_scale: Union[str, list] = 'OrRd',
    layout_size: dict = plotly_default_layout_size, hsplit: int = 1, showlegend=True,
    additional_data: list = [], **kwargs) -> go.Figure:
        
    c_x1, c_y1, c_x2, c_y2 = MultiPoint(geo_data_frame.geometry.centroid).bounds
    center_point = MultiPoint([Point(c_x1, c_y1), Point(c_x2, c_y2)]).centroid
    
    scale_option = dict(range_color=scale_range)
    
    fig = px.choropleth_mapbox(geo_data_frame, geojson=json.loads(geo_data_frame.geometry.to_json()),
        locations='village_code',
        color=scale_column_name,
        hover_name='village_english_name',
        hover_data=['township_english_name'] + additional_data,
        labels={'township_english_name': 'Township Categorization',
                scale_column_name: default_label_format(scale_column_name),
               **{x:default_label_format(x) for x in additional_data}},
        color_continuous_scale=color_scale,
        **scale_option,
        opacity=0.5,
        mapbox_style='carto-positron',
        center={'lon':center_point.x, 'lat':center_point.y},
        zoom=9.5,
        **kwargs
    )
    
    fig.update_traces(hovertemplate=fig['data'][-1]['hovertemplate']\
              .replace('village_code=%{location}<br>','')\
              .replace('=',' = ')\
              .replace('{z}','{z:,.2r}')
    )
    
    if _len_add_data := len(additional_data):
        for x in range(_len_add_data):
            i = x+1
            fig.update_traces(hovertemplate=fig['data'][-1]['hovertemplate']\
                .replace('customdata[{}]'.format(i), 'customdata[{}]:,.2r'.format(i)))
    
    null_data = geo_data_frame.loc[
        geo_data_frame[scale_column_name].isnull(),
        [
            'geometry',
            'village_english_name',
            'township_english_name',
            scale_column_name,
        ]
    ].copy()
    
    fig.add_trace(go.Choroplethmapbox(
        geojson=json.loads(null_data.geometry.to_json()), 
        locations=list(null_data.index),
        z = [0]*len(null_data),
        colorscale=[(0,'grey'),(1,'grey')],
        marker=dict(
            opacity=0.5
        ),      
        hovertext=null_data['village_english_name'],
        customdata=list(map(list, zip(
                null_data['township_english_name']
        ))),
        hovertemplate=fig['data'][-1]['hovertemplate'].replace('%{z:,.2r}','No Data'),
        showscale=False,
        showlegend=False,
    ))
    
    fig.add_trace(go.Choroplethmapbox(
        geojson=taipei_township_geojson, 
        locations=list(taipei_township_gdf.index),
        z = [0]*len(taipei_township_gdf),
        colorscale=[(0,'white'),(1,'white')],
        marker=dict(
            line=(dict(
                color='black',
                width=3,
            )),
            opacity=0.2,
        ),      
        hoverinfo='skip',
        showscale=False,
        showlegend=False,
    ))
    
    add_chart_annotation(fig, '<i>*hover cursor to look at data detail, drag and scroll move</i>')
    
    if not showlegend:
        fig.update_layout(coloraxis_showscale=False)
    
    layout_height = layout_size['height']
    set_layout_size(fig, **{**layout_size, **{'height':layout_height+50}}, hsplit=hsplit)

    fig.update_layout(dict(
        margin={'t':50, 'l':0, 'r':0}
    ))

    return fig


def default_bar_chart_fig(data_frame: pd.DataFrame, x_col: str, y_col: str,
    layout_size: dict = plotly_default_layout_size, hsplit: int = 1, **kwargs) -> go.Figure:
    
    fig = px.bar(data_frame,
        x=x_col,
        y=y_col,
        labels={
            x_col: default_label_format(x_col),
            y_col: default_label_format(y_col)
        },
        **kwargs
    )
    
    fig.update_traces(hovertemplate=fig['data'][-1]['hovertemplate']\
              .replace('{}=%{{x}}<br>'.format(default_label_format(x_col)),'')\
              .replace('=',' = ')\
              .replace('{z}','{z:,.2r}')
    )

    fig.update_layout(showlegend=False)
    fig.update_xaxes(fixedrange=True)
    fig.update_yaxes(fixedrange=True)
    
    set_layout_size(fig, **layout_size, hsplit=hsplit)
    
    fig.update_layout(
        dict(
            margin={'t':50}
        ),
        hovermode='x'
    )
    
    return fig


def default_grouped_bar_chart(left_df: pd.DataFrame, left_x: str, left_y: str, 
    right_df: pd.DataFrame, right_x: str, right_y: str,  layout_size: dict = plotly_default_layout_size,
    hsplit:int = 1) -> go.Figure:

    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name = default_label_format(left_y),
        x = left_df[left_x],
        y = left_df[left_y],
        yaxis= 'y',
        offsetgroup='1'
    ))

    fig.add_trace(go.Bar(
        name = default_label_format(right_y),    
        x = right_df[right_x],
        y = right_df[right_y],
        yaxis = 'y2',
        offsetgroup='2'
    ))

    fig.update_layout(
        barmode='group', 
        xaxis=dict(
            title=default_label_format(left_x),
        ),
        yaxis=dict(
            title=default_label_format(left_y),
            side='left',
        ),
        yaxis2=dict(
            title=default_label_format(right_y),
            overlaying= 'y',
            side='right',
        ),
    )

    fig.update_layout(
        legend=dict(
            orientation='h',
            yanchor='bottom', y=1,
            x=0.15
        ),
        hovermode='x'
    )

    fig.update_xaxes(fixedrange=True)
    fig.update_yaxes(fixedrange=True)
    
    set_layout_size(fig, **plotly_default_layout_size, hsplit=hsplit)

    return fig

def default_show_config(fig: go.Figure) -> None:
    if 'mapbox' in fig._layout.keys():
        fig.show(config=plotly_default_config_geo)
    else:
        fig.show(config=plotly_default_config_chart)

In [ ]:
current_agenda.highlight(0)

# Food industry is the most popular choice for Taiwanese young entrepreneur
---
> "76% of young Taiwanese have either already started their own business or would like to do so... with nearly <span class=h_text>91%</span> of respondents who indicated they wanted to run their own business saying that they are interested in selling street food or local dishes"
> 
> [Taipei Times, 2018](http://www.taipeitimes.com/News/taiwan/archives/2018/06/23/2003695396)

# Entrepreneurship is not easy and good information could help
---
> "... while <span class=h_text>60%</span> who have tried failed to turn a profit."
> 
> [Taipei Times, 2018](http://www.taipeitimes.com/News/taiwan/archives/2018/06/23/2003695396)
<br><br><br>
> "...common reasons new businesses do not make it: <span class=h_text>not knowing your market</span> *enough*, client, competition, buying power..."
>
> [Forbes, 2019](https://www.forbes.com/sites/forbescoachescouncil/2019/07/05/11-reasons-why-most-entrepreneurs-fail/#5accb1dd1b7b)

# This report try to help Taiwanese entrepreneur by informing important information<div class="subtitle">Especially in the food and beverage industry</div>
---
**the term "restaurant" in here mean food and beverage business (could mean food stall or other)*

### This report will cover

- **Restaurant market size**, the number of people that are likely to buy the product
- **Competitor per area**, the number of existed restaurant in the area
- **Cost to open a restaurant in some area**, give estimation of opening a restaurant cost
- **Recommendation for opening a restaurant**, things that could be considered

In [ ]:
current_agenda.highlight(1)

# Taipei Mid-Area have huge market size<div class="subtitle">About 5 times of the other area</div>
---

In [ ]:
taipei_data_gdf['combined_passerby_buying_power'] = \
    taipei_data_gdf['simulated_passerby'] * taipei_data_gdf['passerby_dining_out_daily_budget']

fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'combined_passerby_buying_power', (5000000,70000000))

add_chart_title(fig, "High market size are in Mid-West, Mid-North, and Mid-East",
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, 'source: combination of tax, budget, and simulated passerby')

default_show_config(fig)

# Mid-West is crowded; Mid-North, Mid East pretty crowded<div class="subtitle">The other area have significantly lower passerby number<div>
---

In [ ]:
# left
fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'simulated_passerby', (0,150000),
                                   hsplit=2, showlegend=False)

add_chart_title(fig, "Mid-West area is pretty crowded", 
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, 'source: simulated from goverment Taipei MRT data')

default_show_config(fig)

# right
_agg_col = 'simulated_passerby'
_agg_df = taipei_data_gdf.groupby('township_english_name')[_agg_col].mean().reset_index()
_agg_df.sort_values(_agg_col, ascending=False, inplace=True)

fig = default_bar_chart_fig(_agg_df, 'township_english_name', 'simulated_passerby', hsplit=2)

add_chart_title(fig, "The difference between Mid area <br> to other area crowdedness is significant",
               elevate_line=1.4, title_margin=30, font_size=18)

fig.add_shape(type='rect',
              xref='x', x0=5.6, x1=11.6, yref='paper', y0=0.05, y1=0.7,
              line=dict(
                  color='orange',
                  width=4
              ))

default_show_config(fig)

# The buying power (per person) almost equal everywhere<div class="subtitle">With per meal budget about 200 NTD</div>
---

In [ ]:
fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'passerby_dining_out_daily_budget', (550,750))

add_chart_title(fig, "Almost every area have the same level of buying power",
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, 'source: Taipei tax data and DGBAS data',)

default_show_config(fig)

In [ ]:
current_agenda.highlight(2)

# Mid, Mid-North, and Mid-East, have less competition level<div class="subtitle">Less competition is a good factor to start business</div>
---

In [ ]:
taipei_data_gdf['average_market_share_per_restaurant'] = \
    taipei_data_gdf['combined_passerby_buying_power'] / taipei_data_df['restaurant_count']
taipei_data_gdf['average_market_share_per_restaurant'].replace([np.inf, -np.inf], np.nan, inplace=True)
taipei_data_gdf['average_market_share_per_restaurant'].fillna(0, inplace=True)

fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'average_market_share_per_restaurant', (500000, 2000000), 'RdYlGn')

add_chart_title(fig, "Mid, Mid-North, and Mid-East have lower competition", 
                elevate_line=1.4, title_margin=30, font_size=18)

add_chart_bottom_annotation(fig, 'source: Taipei tax, budget, and travel network data')

default_show_config(fig)

# Most restaurants concentrated around Mid area
---

In [ ]:
fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'restaurant_count', (0, 50))

add_chart_title(fig, 'Mid-West area have large number of restaurants',
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, 'source: simulated from Taipei travel network data')

default_show_config(fig)

# Don't start business in non Mid, Mid-East area<div class="subtitle">The market size is divided with high number of restaurants<div>
---

In [ ]:
# left
_agg_col_1 = 'restaurant_count'
_agg_df_1 = taipei_data_gdf.groupby('township_english_name')[_agg_col_1].sum().reset_index()
_agg_df_1.sort_values(_agg_col_1, ascending=False, inplace=True)

_agg_col_2 = 'combined_passerby_buying_power'
_agg_df_2 = taipei_data_gdf.groupby('township_english_name')[_agg_col_2].sum().reset_index()
_agg_df_2.sort_values(_agg_col_2, ascending=False, inplace=True)

fig = default_grouped_bar_chart(_agg_df_1, 'township_english_name', _agg_col_1,
                               _agg_df_2, 'township_english_name', _agg_col_2,
                               hsplit=2)

fig.add_shape(type='rect',
              xref='x', x0=4.5, x1=6.5, yref='paper', y0=0.0, y1=0.7,
              line=dict(
                  color='orange',
                  width=4
              ))

fig.add_shape(type='rect',
              xref='x', x0=7.5, x1=11.5, yref='paper', y0=0.0, y1=0.7,
              line=dict(
                  color='orange',
                  width=4
              ))

add_chart_title(fig, "Not enough customer per restaurant <br> in outer area", 
                elevate_line=8, title_margin=30, font_size=18)

fig.update_layout(margin={'t':100})

default_show_config(fig)

# right
_agg_col = 'average_market_share_per_restaurant'
_agg_df = taipei_data_gdf.groupby('township_english_name')[_agg_col].mean().reset_index()
_agg_df = _agg_df[~(_agg_df[_agg_col] == np.inf)]
_agg_df.sort_values(_agg_col, ascending=False, inplace=True)

fig = default_bar_chart_fig(_agg_df, 'township_english_name', _agg_col, hsplit=2)

add_chart_title(fig, "The average marketshare per restaurants <br>is low in the outer area", 
                elevate_line=1, title_margin=40, font_size=18)

fig.add_shape(type='rect',
              xref='x', x0=5.5, x1=11.5, yref='paper', y0=0.0, y1=1,
              line=dict(
                  color='orange',
                  width=4
              ))

fig.update_yaxes(dict(title="Market Share per Restaurant"))

default_show_config(fig)


In [ ]:
current_agenda.highlight(3)

# Taipei Mid-North, Mid-East are a good place to start business <div class="subtitle">Decent market size, Less competition, Less rent cost</div>
---

In [ ]:
taipei_data_gdf['simulated_ideal_ROI_index'] = \
    taipei_data_gdf['average_market_share_per_restaurant'] / taipei_data_df['monthly_shop_rent_average_per_ping']
taipei_data_gdf['simulated_ideal_ROI_index'].replace(np.inf, np.nan, inplace=True)

fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'simulated_ideal_ROI_index', (0,1000), 'RdYlGn',
                                   layout_size={'height':400, 'width':600})

add_chart_title(fig, "Mid-North and Mid-West show good ROI", 
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, "source: Taipei tax, budget, restaurant, and rent data")

default_show_config(fig)

# The cost make Mid-West area unlikely profitable<div class="subtitle">They cost about 2 times compare to the other area</div>
---

In [ ]:
# left
taipei_data_gdf['monthly_shop_rent_average_per_ping'].replace(0,np.nan, inplace=True)

fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'monthly_shop_rent_average_per_ping', (1500, 7000),
                                   showlegend=False, hsplit=2)

add_chart_title(fig, "Mid-West area rent cost is enormous", 
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, "source: 591.com (Taiwan's largest Taipei property platform) data")

default_show_config(fig)

# right
_agg_col = 'monthly_shop_rent_average_per_ping'
_agg_df = taipei_data_gdf.groupby('township_english_name')[_agg_col].mean().reset_index()
_agg_df = _agg_df[~(_agg_df[_agg_col] == np.inf)]
_agg_df.sort_values(_agg_col, ascending=False, inplace=True)

fig = default_bar_chart_fig(_agg_df, 'township_english_name', _agg_col, hsplit=2)

add_chart_title(fig, "Mid-West and Mid area have high rent cost", 
                elevate_line=1, title_margin=20, font_size=18)

fig.add_shape(type='rect',
              xref='x', x0=-0.5, x1=1.7, yref='paper', y0=0.0, y1=1,
              line=dict(
                  color='orange',
                  width=4
              ))

default_show_config(fig)

In [ ]:
current_agenda.highlight(4)

# Mid North, Mid East are a good starting place<div class="subtitle">High market size, Less competition, and Decent Rent Cost</div>
---

In [ ]:
# left
filtered_gdf = taipei_data_gdf.loc[
    (taipei_data_gdf['monthly_shop_rent_average_per_ping'] < 3000) & 
    (taipei_data_gdf['simulated_ideal_ROI_index'] > 500),:].copy()
filtered_gdf.sort_values('simulated_ideal_ROI_index', ascending=False, inplace=True)

fig = default_choropleth_mapbox_fig(filtered_gdf, 'simulated_ideal_ROI_index', (0,1000), 'RdYlGn',
                                    showlegend=False, hsplit=2, additional_data=['monthly_shop_rent_average_per_ping'])

add_chart_title(fig, "The place with good ROI is mostly <br>on Mid-North and Mid-East", 
                elevate_line=1.4, title_margin=40, font_size=18)

add_chart_bottom_annotation(fig, "source: Taipei tax, budget, restaurant, and rent data")

default_show_config(fig)

# right
fig = default_grouped_bar_chart(filtered_gdf[:10], 'village_english_name', 'simulated_ideal_ROI_index',
                     filtered_gdf[:10], 'village_english_name', 'monthly_shop_rent_average_per_ping', hsplit=2)

add_chart_title(fig, "Top 10 Villages to <br>start a restaurant", 
                elevate_line=6.3, title_margin=40, font_size=18)

fig.update_layout(margin={'t':100})

default_show_config(fig)

# Sell average price-food in Mid-North, Mid-East<div class="subtitle">Or you could pick the area that fit your product pricing</div>
---

In [ ]:
fig = default_choropleth_mapbox_fig(taipei_data_gdf, 'dining_out_average_monthly_budget', (10000, 40000))

add_chart_title(fig, "Mid-North and Mid-East crowd are not a fancy eater", 
                elevate_line=1.4, title_margin=20, font_size=18)

add_chart_bottom_annotation(fig, 'source: Taipei tax data and DGBAS data',)

default_show_config(fig)

# Consider take away / delivery market<div class="subtitle">The market size is huge, especially in this Covid-19 situation</div>
---
> “We experienced 5,000% growth in the past 24 months,” Yu said. “That’s more than the previous six years combined.”
>
> [Discover Advanced Trends in E-economy Summit, 2020](https://www.taipeitimes.com/News/biz/archives/2020/09/19/2003743674)

# Thank You
---
feel free to reach me via
laurent.saerang@gmail.com